In [ ]:
# default_exp io

# Zarr

<br>

### Imports

In [2]:
import pandas as pd
import xarray as xr
import numpy as np
import dask
import zarr
import gcsfs

In [3]:
james_zarr_bucket = 'solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/OSGB36/all_zarr'

In [4]:
# Override is_retriable. Google Cloud sometimes throws
# a HttpError code 400. gcsfs considers this to not be retriable.
# But it is retriable!

def is_retriable(exception):
    """Returns True if this exception is retriable."""
    errors = list(range(500, 505)) + [
        400, # Jack's addition.  Google Cloud occasionally throws Bad Requests for no apparent reason.
        408,# Request Timeout
        429, # Too Many Requests
    ]
    
    errors += [str(e) for e in errors]
    
    if isinstance(exception, gcsfs.utils.HttpError):
        return exception.code in errors

    return isinstance(exception, gcsfs.utils.RETRIABLE_EXCEPTIONS)

gcsfs.utils.is_retriable = is_retriable

In [5]:
gcs = gcsfs.GCSFileSystem()
james_store = gcsfs.GCSMap(root=james_zarr_bucket, gcs=gcs)
james_xarray = xr.open_zarr(james_store, consolidated=True)

james_xarray

C:\Users\Ayrto\anaconda3\envs\satip_dev\lib\site-packages\google\auth\_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,Array,Chunk
Bytes,3.02 TB,18.52 MB
Shape,"(163137, 704, 548, 12)","(1, 704, 548, 12)"
Count,163138 Tasks,163137 Chunks
Type,float32,numpy.ndarray


<br>

### To Do:

* Reproject all channels not just HRV
* Wrap reprojection functionality inside the `reproj` module
* Succesfully retrieve Zarr data on GCP
* Save data (with correct dtypes) to Zarr on GCP
* Wrap Zarr functions in an `io` module

<br>

### WMS Notes

https://en.wikipedia.org/wiki/Tiled_web_map

Conventions:
* Tiles are 256x256 pixels
* At the outer most zoom level, 0, the entire world can be rendered in a single map tile.
* Each zoom level doubles in both dimensions, so a single tile is replaced by 4 tiles when zooming in. This means that about 22 zoom levels are sufficient for most practical purposes.
* The Web Mercator projection is used, with latitude limits of around 85 degrees.
* An X and Y numbering scheme
* PNG images for tiles
* Images are served through a Web server, with a URL like http://.../Z/X/Y.png, where Z is the zoom level, and X and Y identify the tile.
* Tile Map Service: (0 to 2zoom-1, 2zoom-1 to 0) for the range (-180, +85.0511) - (+180, -85.0511).